In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests
import requests
import pytz
from datetime import datetime as dt


from date_extractor import extract_dates


# Automate Login

In [11]:
# Automate login and scraping

# the URL of the login page
login_url = "https://www.tilastopaja.info/login.php"

# the payload with your login credentials
payload = {
    "user": "SAA",
    "password": "Alexisthe12.",
}

# send the POST request to login
response = requests.post(login_url, data=payload)


# if the request went Ok, you should get a 200 status
print(f"Status code: {response.status_code}")

# parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# find the page title
page_title = soup.title.string

# print the result page title
print(f"Page title: {page_title}")

Status code: 200
Page title: Tilastopaja Login


# Open Directory Files and Process

In [94]:
# LATEST VERSION
# Scrapes Tilastopaja html files

def scraper_new(soup, file):

    year_pattern='20\d\d'
    dash_pattern='\-'

    matches = []
    lists=[]
    counts=[]
    values=[]
    names=[]
    wind=''
    stage=''
    event=''
    sub_event=''
    
    gender=''
    heat=''
    division = ''
    dict_result2=[]  # length 2 dict
    row=[]

    temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'DIVISION', 'REMARKS', 'RX_TIME', 'DICT_RESULTS', 'SOURCE', 'REGION', 'HOST_CITY', 'SUB_EVENT'])

    for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div

        info=div.find('h1')  # then find 'h1' to locate event info

        text=info.text

        list=text.split(',')
        
        # HOST CITY
    
        host_city = list[1]
    
     #   print('HOST CITY', host_city[0])
   
        result=list[0].split(':')

        competition=result[1]
        
      #  competition='Southeast Asian Games'  # Fix the competition for SEAG

      #  print('EVENT', competition)

       # year = re.findall(year_pattern, list[2])

        #print('YEAR', year)

      #  dates = extract_dates(text)

       # year=dates[0].year

        year_pos = re.search(year_pattern, text)

        year = re.findall(year_pattern, text)[0]

      #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

        date1 = re.finditer(dash_pattern, text)


     #   print('YEAR', year)

        for match in date1:

        #    print('MATCH', match.span())

            matches.append(match.span())

        venue =' '

     #   print('MATCHES', matches[0][1])  # choose first tuple, second element

        date = text[matches[0][1]:year_pos.start()]  # extract date

     #   print('DATE', date)


    #h1s = soup.find_all('h1')

    #for h1 in h1s:

    #    text=h1.text

    #    if 'RESULTS' in text:

    #        list=text.split(',')

    #        list1=list[0].split(':')

    #        print('event', list1)

    # VENUE
    
    venue_text = div.find('h2')
    
    venue = venue_text.text
    
   # print('VENUE', venue[7:])



    for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete

        for element in i.find_all('tr'): # <tr> separates each athlete/section

       #     print('NEW SECTION')

           # for td in element.find_all('td'):

            counts=[]
            values=[]


            for count, value in enumerate(element.find_all('td')):

                for n in value.find_all('a'):  # extract the name and put into a list

                    if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           

                        names.append(n.text) # add 'clean' name to list

                for m in value.find_all('b'):  # extract heats/stage/division e.g. U20/U18 etc.

               #     print('NEW HEATS/STAGE/DIVISION', m.text)
            
                    if 'Multievents' in m.text:
                    
                        sub_event = 'Multievents'
                

                    if 'U20' in m.text or 'U18' in m.text or 'U17' in m.text or 'U16' in m.text or 'U15' in m.text or 'U14' in m.text: # From AUS state championships

                        division = m.text
                    else:

                        heat = m.text # NEW ADDITION for Nagase Cup

                    dict_result = ''  # NEW ADDITION for Nagase Cup. Clear dict_result before next heat


                if len(lists)==2:  # captures gender

                   # heat=''
                    wind=''
                    dict_result=''
                    dict_result2=''
                  #  stage=''

               #     print('RESET heat, stage HERE')

                    if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events

                        heat = ''
                        stage = ''

                    if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                        gender = 'Male'

                    if 'Women' in lists[1]:
                        gender = 'Female'


             #   if 'Date' in value.text:
             #       date=value.text

                if 'Wind' in value.text:

                    string=value.text
                    wind=string[5:]  # NEW ADDITION - Nagase Cup - slice out the 'WIND:'
               #     print('WINDDDDD', wind)

         #       print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB

                counts.append(count)
                values.append(value.text)


            lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary


            if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage

                heat = lists[1]
                wind = lists[2]  # wind can also appear in list of length 5

                if 'Wind' in lists[2]:

                    wind = lists[2][5:]  # slice out the 'Wind:'


                dict_result=[]

            if len(lists)==3 and 'Wind' not in lists[2]:


                if 'Semifinals' in lists[1]:

                    stage = 'Semifinals'

                if 'Heats' in lists[1]:

                    stage = 'Heats'

                if 'Heat 1' in lists[1]:

                    heat = 'Heat 1'

                if 'Heat 2' in lists[1]:

                    heat = 'Heat 2'

                if 'Heat 3' in lists[1]:

                    heat = 'Heat 3'

                if 'Heat 4' in lists[1]:

                    heat = 'Heat 4'


              #  if len(lists[2])>1:

              #      date = lists[2]  # set date only if lists[2] is not null

                dict_result = []

            if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered

            #    date = lists[2]
                event = lists[1].strip()
                wind = ' '
                stage = ' '
                heat = ' '

             #   print('Reset STAGE, HEAT here')

                dict_result = []
                dict_result2 = []  # for dictionary of attempts

                if 'Wind' in lists[4]:

                    wind = lists[4][5:]  # slice out the 'Wind:'

               #     print('WINDWIND', wind)


            if len(lists)==11: # when a new athlete is encountered

                dict_result=lists



            #    dict_result2 = [] # reset attempts


            if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists 


            if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists



            # If Long Jump, wind is captured in position 6 of DICT_RESULT #

    
            print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date , 'DIVISION', division, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)

         #   print('DICT_RESULTS', dict_result2)

            
            try:

                if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:



                    if len(dict_result2)>0:

                    #    print('PREVIOUS ROW', temp_df.tail(1))


                       # pass
                      #  temp_df.drop(temp_df.tail(1).index,inplace = True)  # drop previous row without the list of attempts


                 #       temp_df.loc[row_index,'DICT_RESULTS'] = dict_result2



                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])
                        else:
                            row.append(' ')  # name is blank for relays

                        remarks=dict_result[8]+ ' ' + dict_result[2]

                        if 'PB' in remarks:

                            remarks='PB'

                        elif 'SB' in remarks:

                            remarks='SB'

                        if 'Long Jump' in event:  # NEW SECTION July 24-25

                            wind = dict_result[6]

                     #   print('DICT_RESULTS', dict_result2)
                            
                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue[7:])
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB
                        row.append(division)
                        row.append(remarks) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULTS
                        row.append(file)
                        row.append('International')
                        row.append(host_city)
                        row.append(sub_event)



                    elif len(dict_result2)==0:  # append row but do not delete previous row

                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])
                        else:
                             row.append(' ')  # name is blank for relays

                        remarks=dict_result[8]+ ' ' + dict_result[2]

                        if 'PB' in remarks:

                            remarks='PB'

                        elif 'SB' in remarks:

                            remarks='SB'


                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue[7:])
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB
                        row.append(division)
                        row.append(remarks) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULTS
                        row.append(file)
                        row.append('International')
                        row.append(host_city)
                        row.append(sub_event)

                      #  pass




                elif (event and dict_result[3] and dict_result[5]) is not None:

                   #

                    row.append(dict_result[0]) # rank

                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                        row.append(names[-1])  # choose last name in list as the correct one
                    else:
                        row.append(' ')

                    remarks=dict_result[8]+ ' ' + dict_result[2]

                    if 'PB' in remarks:

                        remarks='PB'

                    elif 'SB' in remarks:

                        remarks='SB'


                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue[7:])
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB 
                    row.append(division)
                    row.append(remarks) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULTS
                    row.append(file)
                    row.append('International')
                    row.append(host_city)
                    row.append(sub_event)



            except:

                pass

            if len(row)>5:

           #     print('ADDED ROW to DF', row)


                temp_df.loc[len(temp_df)] = row


           #     print('LENGTH DF', len(temp_df))


         #   print('LENGTH', len(lists))


            dict_result2 = []
            row=[]

        wind=''
        heat=''
   #     date=''


        lists=[]
        dict_result = []
        row=[]

    return temp_df

In [95]:
# LATEST VERSON
# Test open all files in directory

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Asian Athletics Championship/')
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/July/second batch/')


def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items



directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/July/second batch/"
#directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Asian Athletics Championship/"
 
    
    
# Iterate over files in directory

sorted_items = sort_directory(directory)

files = sorted_items

print('number', len(sorted_items))

files_clean = [ file for file in files if file.endswith(".html") ]

#sorted_items.pop(0)  # remove first item (DS_store)

master_df=pd.DataFrame()  # initialize empty master df



for file in files_clean:
    
    print(file)
    
    HTMLFileToBeOpened = open(file, "r") 

    contents = HTMLFileToBeOpened.read() 
    soup = BeautifulSoup(contents, 'lxml') 
    
    temp_df=scraper_new(soup, file)    
    
    
    master_df=pd.concat([master_df, temp_df], axis=0)
    


number 9
Tilastopaja - 32nd Summer Universiade - July 25.html
EVENT  GENDER  STAGE  HEAT  WIND  DATE Wattenscheid, GER  - 21 - 27 July  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT  GENDER  STAGE  HEAT Men WIND  DATE Wattenscheid, GER  - 21 - 27 July  DIVISION  DICT_RESULT  ATTEMPTS {0: '\xa0', 1: 'Men'}
EVENT 100m GENDER Male STAGE   HEAT   WIND  -0.7   DATE Wattenscheid, GER  - 21 - 27 July  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT 100m GENDER Male STAGE   HEAT   WIND  -0.7   DATE Wattenscheid, GER  - 21 - 27 July  DIVISION  DICT_RESULT {0: '1', 1: "Bayanda\xa0WalazaBayanda\xa0Walaza\xa0('06)", 2: '', 3: 'RSA', 4: '9 Feb 06', 5: '10.16 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS []
EVENT 100m GENDER Male STAGE   HEAT   WIND  -0.7   DATE Wattenscheid, GER  - 21 - 27 July  DIVISION  DICT_RESULT {0: '2', 1: "Phooripol\xa0BunsonPhooripol\xa0Bunson\xa0('06)", 2: '', 3: 'THA', 4: '13 Jan 06', 5: '10.22 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS []
EVENT 100m GENDER

EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE  5 - 6 July  DIVISION  DICT_RESULT {0: '34', 1: "Samuel\xa0TollSamuel\xa0Toll\xa0('98)", 2: '', 3: 'AUS', 4: '28 Jan 98', 5: '2:25:30 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} ATTEMPTS []
EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE  5 - 6 July  DIVISION  DICT_RESULT {0: '35', 1: "Nur\xa0ShodiqNur\xa0Shodiq\xa0('92)", 2: '', 3: 'INA', 4: '17 May 92', 5: '2:26:09 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} ATTEMPTS []
EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE  5 - 6 July  DIVISION  DICT_RESULT {0: '36', 1: "Jordan\xa0SkellyJordan\xa0Skelly\xa0('92)", 2: '', 3: 'GBR', 4: '31 Jul 92', 5: '2:26:32 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS []
EVENT Marathon     GENDER Male STAGE   HEAT   WIND   DATE  5 - 6 July  DIVISION  DICT_RESULT {0: '37', 1: "Jacob\xa0FosterJacob\xa0Foster\xa0('91)", 2: '', 3: 'AUS', 4: '18 Nov 91', 5: '2:26:47 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} 

HOST CITY  
VENUE events: 
EVENT  GENDER  STAGE  HEAT  WIND  DATE  5 July  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT  GENDER  STAGE  HEAT Men WIND  DATE  5 July  DIVISION  DICT_RESULT  ATTEMPTS {0: '\xa0', 1: 'Men'}
EVENT Pole Vault     GENDER Male STAGE   HEAT   WIND   DATE  5 July  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT Pole Vault     GENDER Male STAGE   HEAT   WIND   DATE  5 July  DIVISION  DICT_RESULT {0: '1', 1: "Koen\xa0van der WijstKoen\xa0van der Wijst\xa0('97)", 2: '', 3: 'NED', 4: '7 Aug 97', 5: '5.44 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS []
EVENT Pole Vault     GENDER Male STAGE   HEAT   WIND   DATE  5 July  DIVISION  DICT_RESULT {0: '1', 1: "Koen\xa0van der WijstKoen\xa0van der Wijst\xa0('97)", 2: '', 3: 'NED', 4: '7 Aug 97', 5: '5.44 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS {0: '\xa0', 1: '5.10/1      5.30/1      5.44/1      5.51/XXX      '}
EVENT Pole Vault     GENDER Male STAGE  HEAT  WIND  DATE  5 July  DIVISION  DICT_RESULT {0: '

In [96]:
master_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,...,WIND,DOB,DIVISION,REMARKS,RX_TIME,DICT_RESULTS,SOURCE,REGION,HOST_CITY,SUB_EVENT
0,1,Bayanda Walaza,RSA,10.16,,32nd Summer Universiade,2025,"Wattenscheid, GER - 21 - 27 July",100m,Lohrheidestadion,...,-0.7,9 Feb 06,,« »,,[],Tilastopaja - 32nd Summer Universiade - July 2...,International,Bochum-Wattenscheid,
1,2,Phooripol Bunson,THA,10.22,,32nd Summer Universiade,2025,"Wattenscheid, GER - 21 - 27 July",100m,Lohrheidestadion,...,-0.7,13 Jan 06,,« »,,[],Tilastopaja - 32nd Summer Universiade - July 2...,International,Bochum-Wattenscheid,
2,3,Hiroki Yanagita,JPN,10.23,,32nd Summer Universiade,2025,"Wattenscheid, GER - 21 - 27 July",100m,Lohrheidestadion,...,-0.7,25 Jul 03,,« »,,[],Tilastopaja - 32nd Summer Universiade - July 2...,International,Bochum-Wattenscheid,
3,4,Muhd Azeem Fahmi,MAS,10.35,,32nd Summer Universiade,2025,"Wattenscheid, GER - 21 - 27 July",100m,Lohrheidestadion,...,-0.7,29 Apr 04,,« »,,[],Tilastopaja - 32nd Summer Universiade - July 2...,International,Bochum-Wattenscheid,
4,5,Kyle Zinn,RSA,10.42,,32nd Summer Universiade,2025,"Wattenscheid, GER - 21 - 27 July",100m,Lohrheidestadion,...,-0.7,14 Jan 03,,« »,,[],Tilastopaja - 32nd Summer Universiade - July 2...,International,Bochum-Wattenscheid,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2,Rick Schipper,NED,59.99,,Track Meeting Eindhoven - Memorial Cor Aalberts,2025,"Memorial Cor Aalberts, Eindhoven, NED - 19 J...",Hammer Throw (6kg),Sportpark De Hondsheuvels,...,,,U20,PB,,[],Tilastopaja - Track Meeting Eindhoven - Memori...,International,Eindhoven,
98,2,Rick Schipper,NED,59.99,,Track Meeting Eindhoven - Memorial Cor Aalberts,2025,"Memorial Cor Aalberts, Eindhoven, NED - 19 J...",Hammer Throw (6kg),Sportpark De Hondsheuvels,...,,,U20,PB,,"{0: ' ', 1: ' 57.89 57.34 59.99...",Tilastopaja - Track Meeting Eindhoven - Memori...,International,Eindhoven,
99,4,Ruben Driessens,NED,4.50,,Track Meeting Eindhoven - Memorial Cor Aalberts,2025,"Memorial Cor Aalberts, Eindhoven, NED - 19 J...",Pole Vault,Sportpark De Hondsheuvels,...,,,U18,PB,,[],Tilastopaja - Track Meeting Eindhoven - Memori...,International,Eindhoven,
100,4,Ruben Driessens,NED,4.50,,Track Meeting Eindhoven - Memorial Cor Aalberts,2025,"Memorial Cor Aalberts, Eindhoven, NED - 19 J...",Pole Vault,Sportpark De Hondsheuvels,...,,,U18,PB,,"{0: ' ', 1: '4.00/2 4.10/1 4.20/1 ...",Tilastopaja - Track Meeting Eindhoven - Memori...,International,Eindhoven,


In [97]:
master_df['DICT_RESULTS']

0                                                     []
1                                                     []
2                                                     []
3                                                     []
4                                                     []
                             ...                        
97                                                    []
98     {0: ' ', 1: '  57.89        57.34        59.99...
99                                                    []
100    {0: ' ', 1: '4.00/2      4.10/1      4.20/1   ...
101                                                   []
Name: DICT_RESULTS, Length: 2603, dtype: object

In [98]:
# Remove special characters
    
for col in master_df.columns:
    master_df[col] = master_df[col].astype(str)
    master_df[col] = master_df[col].str.replace('\xa0', ' ', regex=True)
    master_df[col] = master_df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    master_df[col] = master_df[col].str.replace('\r', ' ', regex=True)
    master_df[col] = master_df[col].str.replace('\n', ' ', regex=True)
    master_df[col] = master_df[col].str.strip()


    


In [99]:
# removes duplicate rows from those with list of attempts

master_df = master_df.drop_duplicates(['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'HOST_CITY'], keep='last')

In [100]:
# removes duplicate rows from those with list of attempts for relays

master_df = master_df.drop_duplicates(['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'HOST_CITY'], keep='last')

In [101]:
# Map general event category 

master_df.loc[master_df['EVENT'].str.contains(r'^100m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^400m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^600m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^60m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^200m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'50m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'80m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'300m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^3000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'Mile', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^5000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'^2000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^1000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^800m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^1500m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'10,000m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'5km', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'10km', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

master_df.loc[master_df['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
master_df.loc[master_df['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
master_df.loc[master_df['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
master_df.loc[master_df['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
master_df.loc[master_df['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
master_df.loc[master_df['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
master_df.loc[master_df['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Heptathlon', na=True), 'CATEGORY_EVENT'] = 'Heptathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
master_df.loc[master_df['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
master_df.loc[master_df['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'

# Road race

master_df.loc[master_df['EVENT'].str.contains(r'Road|road', na=True), 'CATEGORY_EVENT'] = 'Road'


# Copy relay event dictionary names into NAME column

master_df['NAME'] = np.where(((master_df['NAME'] == '') & ((master_df['EVENT']=='4 x 100m')|(master_df['EVENT']=='4 x 400m'))), master_df['DICT_RESULTS'], master_df['NAME'])

In [102]:
# Convert to NEW SCHEMA



master_df['LAST_NAME'] = ''
master_df['FIRST_NAME'] = ''
master_df['OTHER_NAME'] = ''
master_df['TEAM'] = ''
master_df['SEED'] = ''
master_df['LANE'] = ''
master_df['DIVISION'] = ''
master_df['AGE'] = ''
master_df['UNIQUE_ID'] = ''
master_df['ATHLETE_ID'] = ''
master_df['TIMESTAMP'] = ''
master_df['TAG_ID'] = ''
master_df['POINTS'] = ''
master_df['GROUP'] = ''
master_df['SESSION']=''
master_df['EVENT_CLASS']=''
master_df['DISTANCE']=''


master_df = master_df.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


In [103]:
SGP_athletes=master_df[master_df['NATIONALITY']=='SGP']

SGP_athletes.reset_index(drop=True, inplace=True)


In [104]:
df = SGP_athletes

In [105]:
# Check file before saving
gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
#df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)
#df['event_date_dt'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, utc=True)

stage_list = ['Finals', 'Final', 'Semifinal', 'Heats', 'Qualification', '']

event_list = ['100m', '400m', '800m', '1500m', '1500m Walk', '3000m', '5000m', '10,000m', '80m Hurdles', '110m Hurdles', '100m Hurdles', '200m Hurdles', '400m Hurdles', 
              'High Jump', 'Discus', 'Discus Throw', 'Triple Jump', 'Pole Vault','Long Jump', 'Javelin', 'Javelin Throw', 'Shot Put', 'High Jump', '2000m SC', '3000m Walk', 
              '5000m Walk', 'Hammer Throw', '200m', '3000m SC', '4 x 100m Relay', '4 x 400m Relay'] 
category_list = ['Throw', 'Jump', 'Relay', 'Long', 'Mid', 'Sprint', 'Road', 'Cross Country', 'Steeple', 'Hurdles', 'Walk', 'Marathon', 'Triathlon', 'Decathlon', 'Pentathlon']

for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
#    assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] in category_list
    assert len(row['DATE'])>3
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
    assert row['NATIONALITY']=='SGP' # For Tilastopaja extractions
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    assert row['STAGE'] in stage_list 
    assert row['EVENT'] in event_list 
    assert re.search('Para|para', row['EVENT']) is None
    assert re.search('events', row['VENUE']) is None
    

    
 #   assert not row['event_date_dt'] is pd.NaT
    
    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

    # Set timestamp

    timezone = pytz.timezone('UTC')
    dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
    df.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)


print('PASSED')

df

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92012/987823273.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['YEAR'] = df['YEAR'].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92012/987823273.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE'] = df['DATE'].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92012/987823273.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

AssertionError: 

In [106]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/July/second batch/')

df.to_csv("SGP_athletes_July_2025_second.csv", index=False, encoding="utf-8")

# Read and process manually cleaned data

In [753]:
# read manually cleaned data

SGP_athletes = pd.read_csv("SGP_athletes_cleaned.csv")

In [754]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT,SOURCE,REGION
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,Male,,,,26 Feb 96,« »,,[],Tilastopaja - UAE Athletics Grand Prix - May ...,International
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,Female,,,,27 Nov 92,« »,Mixed competition,[],Tilastopaja - 11. Münchner Abendsportfest :MU...,International
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,Male,Heats,Heat 1,,3 Jul 94,SB «,,[],Tilastopaja - 11th Asian Indoor Athletics Cham...,International
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,Male,,,,26 Feb 96,SB «,,[],Tilastopaja - 11th Kinami Michitaka Memorial A...,International
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,Male,Heats,Heat 2,,26 May 01,« »,0.187,[],Tilastopaja - 11th Para Athletics World Champi...,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,Male,Heats,Heat 2,,26 Feb 96,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Male,NaN,NaN,NaN,26 Apr 06,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International


In [755]:
# Map general event category 

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'




In [756]:
# Convert to OLD SCHEMA

#SGP_athletes['TEAM'] = 'Singapore'
#SGP_athletes['SEED'] = ''
#SGP_athletes['LANE'] = ''
#SGP_athletes['DIVISION'] = ''
#SGP_athletes['AGE'] = ''
#SGP_athletes['UNIQUE_ID'] = ''
#SGP_athletes['ATHLETE_ID'] = ''
#SGP_athletes['TIMESTAMP'] = ''
#SGP_athletes['TAG_ID'] = ''
#SGP_athletes['POINTS'] = ''
#SGP_athletes['GROUP'] = ''
#SGP_athletes['FREE_FIELD']=''
#SGP_athletes['FREE_FIELD2']=''
#SGP_athletes['FREE_FIELD3']=''


In [ ]:
# Convert to NEW SCHEMA

SGP_athletes['LAST_NAME'] = ''
SGP_athletes['FIRST_NAME'] = ''
SGP_athletes['OTHER_NAME'] = ''
SGP_athletes['TEAM'] = 'Singapore'
SGP_athletes['SEED'] = ''
SGP_athletes['LANE'] = ''
SGP_athletes['DIVISION'] = ''
SGP_athletes['AGE'] = ''
SGP_athletes['UNIQUE_ID'] = ''
SGP_athletes['ATHLETE_ID'] = ''
SGP_athletes['TIMESTAMP'] = ''
SGP_athletes['TAG_ID'] = ''
SGP_athletes['POINTS'] = ''
SGP_athletes['GROUP'] = ''
SGP_athletes['VENUE']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['SESSION']=''
SGP_athletes['EVENT_CLASS']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['DISTANCE']=''
SGP_athletes['RX_TIME']=''
SGP_athletes['CATEGORY_EVENT']=''


In [757]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,,,,,,,,,,


In [758]:
df = SGP_athletes.drop(['VENUE', 'YEAR', 'RX_TIME'], axis=1)
df['DATE']='2024'

In [759]:
df

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,DATE,EVENT,GENDER,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,400m Hurdles,Male,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,5000m,Female,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,60m Hurdles,Male,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,400m Hurdles,Male,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,400m,Male,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,400m Hurdles,Male,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Male,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Female,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,Triple Jump,Female,...,,,,,,,,,,


In [760]:
# Re-order columns

df = df.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


In [51]:
df

NameError: name 'df' is not defined

In [762]:
df.columns

Index(['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
       'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE',
       'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION',
       'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID', 'SOURCE',
       'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'],
      dtype='object')

In [763]:
# Remove special characters
    
for col in df.columns:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()


    
    

In [764]:
df.reset_index(inplace=True)

In [50]:
# Check file before saving

ct = dt.now()
gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)
#df['event_date_dt'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, utc=True)


for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
    assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] is not None
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
    assert len(row['DATE'])>3
    assert row['NATIONALITY']=='SGP'
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

    # Set timestamp

    df.loc[:,'TIMESTAMP'] = ct


print('PASSED')

df

NameError: name 'df' is not defined

In [765]:
df.to_csv('SG_athletes_international_2024.csv', encoding='utf-8')

# Process missing data extracted from SAA database

In [237]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/External events/')

athletes = pd.read_csv("ALL_external_data_from_SAA_database_Jan 2011_to_Dec_24.csv")

In [238]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any"
0,19 Dec 2024,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-
1,19 Dec 2024,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-
2,07 Dec 2024,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-
3,07 Dec 2024,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-
4,07 Dec 2024,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-
...,...,...,...,...,...,...,...,...,...,...
6535,16 Apr 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne"
6536,27 Mar 2011,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee..."
6537,27 Mar 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne"
6538,11 Mar 2011,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand"


In [239]:
# Remove special characters
    
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()



In [240]:
athletes['REGION'] = 'International' 
athletes['SOURCE'] = 'SAA Database'



In [241]:
# convert Date column into datetime

athletes['Date']=pd.to_datetime(athletes['Date'], format='mixed')

athletes['Date']

0      2024-12-19
1      2024-12-19
2      2024-12-07
3      2024-12-07
4      2024-12-07
          ...    
6535   2011-04-16
6536   2011-03-27
6537   2011-03-27
6538   2011-03-11
6539   2011-02-20
Name: Date, Length: 6540, dtype: datetime64[ns]

In [242]:
# extract day and month only

athletes['date_extract']=athletes['Date'].dt.strftime('%d-%m')
athletes['YEAR']=athletes['Date'].dt.strftime('%Y')

In [243]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011


In [244]:
mask = athletes['Event'].str.contains(r'Men', na=True)
athletes.loc[mask, 'GENDER'] = 'Male'

mask = athletes['Event'].str.contains(r'Women', na=True)
athletes.loc[mask, 'GENDER'] = 'Female'


In [245]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male


In [246]:
athletes['COMPETITION'] = athletes['Competition'].str[16:]  # Remove 'International -' from string

In [247]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia)
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia)
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships


In [248]:
mask = athletes['Info, if any'].str.contains(r'Final', na=True)
athletes.loc[mask, 'STAGE'] = 'Final'

mask = athletes['Info, if any'].str.contains(r'Prelim', na=True)
athletes.loc[mask, 'STAGE'] = 'Prelim'

mask = athletes['Info, if any'].str.contains(r'Semifinal', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'

mask = athletes['Info, if any'].str.contains(r'Semi', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'


mask = athletes['Info, if any'].str.contains(r'Heats', na=True)
athletes.loc[mask, 'STAGE'] = 'Heats'


In [249]:
mask = athletes['Info, if any'].str.contains(r'U20', na=True)
athletes.loc[mask, 'DIVISION'] = 'U20'

mask = athletes['Info, if any'].str.contains(r'U18', na=True)
athletes.loc[mask, 'DIVISION'] = 'U18'


In [250]:
'''

mask = athletes['Info, if any'].str.contains(r'PB', na=True)
athletes.loc[mask, 'REMARKS'] = 'PB'

mask = athletes['Info, if any'].str.contains(r'SB', na=True)
athletes.loc[mask, 'REMARKS'] = 'SB'

mask = athletes['Info, if any'].str.contains(r'CR', na=True)
athletes.loc[mask, 'REMARKS'] = 'CR'

mask = athletes['Info, if any'].str.contains(r'NR', na=True)
athletes.loc[mask, 'REMARKS'] = 'NR'

mask = athletes['Info, if any'].str.contains(r'R1', na=True)
athletes.loc[mask, 'REMARKS'] = 'R1'

'''


"\n\nmask = athletes['Info, if any'].str.contains(r'PB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'PB'\n\nmask = athletes['Info, if any'].str.contains(r'SB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'SB'\n\nmask = athletes['Info, if any'].str.contains(r'CR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'CR'\n\nmask = athletes['Info, if any'].str.contains(r'NR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'NR'\n\nmask = athletes['Info, if any'].str.contains(r'R1', na=True)\nathletes.loc[mask, 'REMARKS'] = 'R1'\n\n"

In [251]:
athletes.rename(columns={'Event': 'EVENT'}, inplace=True)


In [252]:
# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'10 Kilometer Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'


mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'
mask = athletes['EVENT'].str.contains(r'Mile', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'


mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'



# Hurdles

mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^100m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = athletes['EVENT'].str.contains(r'^100\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'^110m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


#mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
#mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'



mask = athletes['EVENT'].str.contains(r'^200m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200m\sHurdles\s\(0\.762m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400m\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


mask = athletes['EVENT'].str.contains(r'400m\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'^Javelin\sThrow$', na=True, regex=True)  # exact match for string
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'



mask = athletes['EVENT'].str.contains(r'^Shot\sPut$', na=True, regex=True) # there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4\.00kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'^Shot\sput$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Shot Put Masters', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'^Hammer\sThrow$', na=True)  # there are some characters after Throw
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'

mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'

#mask = athletes['EVENT'].str.contains(r'^Discus\sThrow    $', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sthrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=True))  & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



mask = athletes['EVENT'].str.contains(r'Women\sDiscus\sThrow\s\(1kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOPEN', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'Long\sJump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'

# Marathon

mask = athletes['EVENT'].str.contains(r'Marathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'Half\sMarathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half marathon'

# Walk

mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m Racewalk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m Relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'

# Cross Country

mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10km Cross Country'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_13049/745616938.py:340: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)


In [253]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,800m
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,1500m
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk


In [254]:
# Extract ranking

def rank(string):
    
    

    try:
        
        slash = re.search(r'\/', string) 
        
        d1 = re.search(r'^\dpos', string)
        d2 = re.search(r'^\d\dpos', string)
        d3 = re.search(r'^\d\d\d\dpos', string)
        d4 = re.search(r'^\d\d\dpos', string)
        
        print(d1, d2, d3, d4)
        
        print(slash)
        
        if slash:
     
            start = slash.start()-2
    
            end = slash.start()
    
            pos = string[start:end]
        
        elif d1:
            
            pos = string[:1]
            
            print(d1)
            
        elif d2:
            
            pos = string[:2]
            
            print(d2)
            
        elif d3:
            
            pos = string[:4]
            
        elif d4:
            
            pos = string[:3]
            
            
        else:
            
            pos=''
            
    except:
        
        pos = ''
    
    return pos
    

athletes['RANK'] = athletes['Info, if any'].apply(rank)


None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; 

In [6]:
# Extract date

def date(string):
    
    
    try:
        
        slash=re.search(r'2025', string)
        
        end = slash.start()
    
        date = string[:end]
            
    except:
        
        date = ''
    
    return date
    

#athletes['DATE'] = athletes['Date'].apply(date)


In [7]:
athletes

NameError: name 'athletes' is not defined

In [257]:
# Extract heat

def heat(string):
    
    
    try:
        
        heat=re.findall(r'h\d|H\d|h\d\d|H\d\d', string)
        
        heat1=heat[0]
                
            
    except:
        
        heat1 = ''
    
    return heat1
    

athletes['HEAT'] = athletes['Info, if any'].apply(heat)


In [258]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT,RANK,HEAT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,800m,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,1500m,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put,,
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m,,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put,,
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk,,


In [5]:
# Extract qualitication

def qual(string):
    
    
    try:
        
        qual=re.findall(r'\sq|\sQ', string)
        qual1=qual[0]
        
                
            
    except:
        
        qual1 = ''
    
    return qual1
    

athletes['QUALIFICATION'] = athletes['Info, if any'].apply(qual)


NameError: name 'athletes' is not defined

In [260]:
# Extract event class

def event_class(string):
    
    
    try:
        
        left=re.search(r'\(', string)
        right=re.search(r'\)', string)
        
        
        type = string[left.start()+1:right.start()]
        
                 
    except:
        
        type = ''
    
    return type
    

athletes['EVENT_CLASS'] = athletes['EVENT'].apply(event_class)


In [261]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,2024,Male,Strive (Australia),NaN,NaN,800m,,,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,2024,Male,Vic Milers (Australia),NaN,NaN,1500m,,,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m,,,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put,,,,7.26kg
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m,,,Q,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put,,,,7.26kg
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk,,,,


In [262]:
athletes.to_csv("SAA_external_database.csv", encoding="utf-8")

In [263]:
athletes = athletes.rename(columns={'Info, if any': 'REMARKS', 'Wind m/s': "WIND", 'Age': "AGE", 'Result': "RESULT", 'date_extract': "DATE", 'Year D.O.B.': "DOB"})

In [264]:
athletes = athletes.drop(['Date', 'Competition', 'EVENT'], axis=1)

In [265]:
athletes = athletes.rename(columns={'MAPPED_EVENT': "EVENT", 'Name': "NAME", 'Team': "TEAM"})

In [266]:
athletes = athletes[athletes['YEAR']=='2023']

In [267]:
athletes

,NAME,AGE,TEAM,RESULT,WIND,DOB,REMARKS,REGION,SOURCE,DATE,YEAR,GENDER,COMPETITION,STAGE,DIVISION,EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
530,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.01,-0.2,2004,Heats 1/8pos q,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Heats,NaN,100m,1,,q,
531,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.08,-0.1,2004,Final 8/8pos,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,100m,8,,,
532,"MUTHUKUMARAN, RAAM KUMAR",19,Singapore Sports School,3:20.75,-,2004,Final 1/8pos,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,4 x 400m Relay,1,,,
533,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,22.32,-0.3,2004,Final 3/8pos,International,SAA Database,19-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,200m,3,,,
534,"YUSRAN, NIQ RIFDI RAFIQI",16,Singapore Sports School,55.05,-,2007,Heats 4/8pos,International,SAA Database,19-12,2023,Male,SAECA-BJSS Athletics Championship,Heats,NaN,400m,4,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,"ROZARIO, TIA LOUISE",23,PRINCETON,5.52,Indoor,2000,"5/8pos, sanctioned",International,SAA Database,28-01,2023,Female,Harvard-Yale-Princeton,NaN,NaN,Long Jump,,,,
2115,"KAM, KAMPTON",22,INDIVIDUAL,2.08,-,2001,"Sanctioned,1pos",International,SAA Database,21-01,2023,Male,Wesley A Brown Invitational,NaN,NaN,High Jump,,,,
2116,"PHUA, JASMIN",22,Individual,42.83,-,2000,-,International,SAA Database,20-01,2023,Female,Strive Season Program B,NaN,NaN,Discus Throw,,,,1kg
2117,"KAM, KAMPTON",22,Individual,2.06,-,2000,"Sanctioned,1pos",International,SAA Database,14-01,2023,Male,Penn 10-Team Select,NaN,NaN,High Jump,,,,


In [268]:
athletes.to_csv("athletes_2023.csv", encoding='utf-8')

In [269]:
# Map general event category 

athletes.loc[athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'mile', na=True), 'CATEGORY_EVENT'] = 'Mid'

athletes.loc[athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

athletes.loc[athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'vault', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'jump', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
athletes.loc[athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
athletes.loc[athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
athletes.loc[athletes['EVENT'].str.contains(r'hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'

athletes.loc[athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'Steeplechase', na=True), 'CATEGORY_EVENT'] = 'Steeple'




In [270]:
# Extract date

def time(string):
    
    
    try:
        
        slash=re.search(r'AM|PM', string)
        
        end = slash.start()
    
        time = string[:end]
            
    except:
        
        time = ''
    
    return time
    

#athletes['RESULT_CLEANED'] = athletes['RESULT'].apply(time)


In [271]:
athletes.columns

Index(['NAME', 'AGE', 'TEAM', 'RESULT', 'WIND', 'DOB', 'REMARKS', 'REGION',
       'SOURCE', 'DATE', 'YEAR', 'GENDER', 'COMPETITION', 'STAGE', 'DIVISION',
       'EVENT', 'RANK', 'HEAT', 'QUALIFICATION', 'EVENT_CLASS',
       'CATEGORY_EVENT'],
      dtype='object')

In [272]:
# Convert to NEW SCHEMA

athletes['LAST_NAME'] = ''
athletes['FIRST_NAME'] = ''
athletes['OTHER_NAME'] = ''
athletes['SEED'] = ''
athletes['LANE'] = ''
athletes['ATHLETE_ID'] = ''
athletes['TIMESTAMP'] = ''
athletes['TAG_ID'] = ''
athletes['POINTS'] = ''
athletes['GROUP'] = ''
athletes['SESSION']=''
athletes['DISTANCE']=''
athletes['HOST_CITY']=''
athletes['SUB_EVENT']=''
athletes['DICT_RESULTS']=''
athletes['RX_TIME']=''






athletes = athletes.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


In [273]:
Marathon_2023 = athletes[athletes['EVENT']=='Marathon']

In [275]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


Marathon_2023.to_csv("SAA_database_marathon_2023.csv", encoding='utf-8')

In [56]:
athletes

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,,,,NaN,,,NaN,,1:53.51,,...,SAA Database,NaN,,NaN,,,,,,
1,,,,NaN,,,NaN,,4:01.49,,...,SAA Database,NaN,,NaN,,,,,,
2,,,,NaN,,,NaN,,6.84,,...,SAA Database,NaN,,NaN,,,,,,
3,,,,NaN,,,NaN,,21.68,,...,SAA Database,NaN,,NaN,,,,,,
4,,,,NaN,,,NaN,,6.97,,...,SAA Database,NaN,,NaN,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,,,,NaN,,,NaN,,14.91,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6536,,,,NaN,,,NaN,,37:15.87,Q,...,SAA Database,NaN,,NaN,,,,,,
6537,,,,NaN,,,NaN,,15.32,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6538,,,,NaN,,,NaN,,26:53,,...,SAA Database,NaN,,NaN,,,,,,
